# Bats data selection randomization 

Import libraries

In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import random
import warnings

# Suppress the specific warning
warnings.filterwarnings("ignore", category=Warning)

Read data

In [2]:
df1 = pd.read_csv('ubna_data_01_collected_audio_records.csv', index_col=0)
df2 = pd.read_csv('ubna_data_02_collected_audio_records.csv', index_col=0)
df3 = pd.read_csv('ubna_data_03_collected_audio_records.csv', index_col=0)

df = pd.concat([df1, df2, df3], ignore_index=True)

# print(df['Datetime UTC'].dtype)
df['Datetime UTC'] = pd.to_datetime(df['Datetime UTC'])
# print(df['Datetime UTC'].dtype)

df2022 = df.loc[df['Datetime UTC'].dt.year == 2022]
#print(df2022['Datetime UTC'].dt.month.unique())
#print(df2022['Site name'].unique())
df2022 = df2022[df2022['Site name'] != '(Site not found in Field Records)']

df2022 = df2022.reset_index(drop=True)
#display(df2022)


In [3]:
dfw = pd.read_csv('2022-uwa.csv', header=[1])

# Convert 'Date' column to datetime format
dfw['Datetime UTC'] = pd.to_datetime(dfw['Date'] + ' ' + dfw['Time'])

dfw = dfw.loc[:, ['Datetime UTC', 'Rain', 'Speed']]

#display(dfw)


Since the length of each measurement in the two databases is not the same, here, I take the time interval of the recorder as the basis, and take the weather data in the recorder time interval to take the mean (for the wind speed) and sum (for the precipitation) and merge them into the recorder database.

Therefore, the database generated is the average wind speed and total rainfall from the last record time to this record time.

In [4]:
unique_recording_date = df2022['Datetime UTC'].unique()

save_mean = pd.DataFrame()

for i in range(len(unique_recording_date) - 1):
    start_date = unique_recording_date[i]
    end_date = unique_recording_date[i+1]
    sum_rain = dfw.loc[(dfw['Datetime UTC'] >= start_date) & (dfw['Datetime UTC'] <= end_date), 'Rain'].sum()
    mean_speed = dfw.loc[(dfw['Datetime UTC'] >= start_date) & (dfw['Datetime UTC'] <= end_date), 'Speed'].mean()
    result = {'Datetime UTC': end_date, 'Sum Rain': sum_rain, 'Mean Speed': mean_speed}
    save_mean = save_mean.append(result, ignore_index=True)

display(save_mean)

,Datetime UTC,Sum Rain,Mean Speed
0,2022-06-14 23:00:00,0.0,9.720833
1,2022-06-14 23:30:00,0.0,10.070968
2,2022-06-15 00:00:00,0.0,9.512903
3,2022-06-15 00:30:00,0.0,9.032258
4,2022-06-15 01:00:00,0.0,9.287097
...,...,...,...
27953,2022-12-23 00:03:02,0.0,NaN
27954,2022-12-23 00:03:04,0.0,NaN
27955,2022-12-23 00:03:06,0.0,NaN
27956,2022-12-23 00:03:08,0.0,NaN


Check the max / min in the dataset to set the bar for "heavy wind" and "heavy rain"

In [5]:
print(save_mean['Sum Rain'].min())
print(save_mean['Sum Rain'].max())
print(save_mean['Mean Speed'].min())
print(save_mean['Mean Speed'].max())
#display(save_mean[save_mean['Sum Rain']> 0.02])
#display(save_mean[save_mean['Mean Speed']>15])

0.0
1.57
2.9
18.987096774193553


Merge the generate dataset and the recorder dataset together.

In [6]:
combined_df = pd.merge(df2022, save_mean, on='Datetime UTC')
#display(combined_df)

 Select 2 rows for different location each month. Extreme weather conditions are defined as 'Mean Speed' exceeding 15 or 'Sum Rain' surpassing 0.02. If these conditions are met, the selection process will ensure that at least one row represents extreme weather conditions, while the other row represents normal weather conditions. Otherwise, the program will select 2 row from the normal weather condition. The selected rows will be meticulously and randomly chosen from the dataset. But here, to be able to reproduce my code, I set up random state = 1.

In [7]:
combined_df['Month'] = combined_df['Datetime UTC'].dt.month

def check_extreme_conditions(combined_df):
    return combined_df['Mean Speed'] > 15 or combined_df['Sum Rain'] > 0.02

# Initialize an empty DataFrame to store the selected rows
selected_rows = pd.DataFrame()

grouped_data = combined_df.groupby(['Site name', 'Month'])

# Iterate through each group to select rows
for group_name, group_data in grouped_data:
    extreme_weather_data = group_data[group_data.apply(check_extreme_conditions, axis=1)]
    normal_weather_data = group_data[~group_data.apply(check_extreme_conditions, axis=1)]

    if len(normal_weather_data) >=2:
        if len(extreme_weather_data) >=1:
            selected_extreme_row = extreme_weather_data.sample(n=1, random_state=1)
            selected_normal_row = normal_weather_data.sample(n=1, random_state=1)
            selected_rows = selected_rows.append(selected_extreme_row).append(selected_normal_row)
    
        else:
            selected_normal_row = normal_weather_data.sample(n=2, random_state=1)
            selected_rows = selected_rows.append(selected_normal_row)

# Display the selected rows
print("Selected Rows:")
display(selected_rows)


Selected Rows:


,Datetime UTC,Site name,Recover folder,AudioMoth #,SD card #,File path,File metadata,File status,Audiomoth temperature,Audiomoth battery,Sample rate,Audiomoth artist ID,File duration,Deployment notes,Sum Rain,Mean Speed,Month
7884,2022-07-19 06:30:00,Carp Pond,recover-20220719,F,004,/mnt/ubna_data_01/recover-20220719/UBNA_004/20...,Recorded at 06:30:00 19/07/2022 (UTC) by Audio...,Usable for detection,21.8C,3.8V,250000,AudioMoth 249BC30461CBEB1A,1795,Case,0.00,5.041935,7
9163,2022-07-28 01:00:00,Carp Pond,recover-20220728,C,007,/mnt/ubna_data_01/recover-20220728/UBNA_007/20...,Recorded at 01:00:00 28/07/2022 (UTC) by Audio...,Usable for detection,36.9C,4.0V,250000,AudioMoth 24F319055FDF2F5B,1795,Double Battery Pack; Ziploc Bag,0.00,7.516129,7
13304,2022-08-23 00:00:00,Carp Pond,recover-20220822,C,012,/mnt/ubna_data_01/recover-20220822/UBNA_012/20...,Is empty!,Not usable; no metadata,File does not have metadata!,File does not have metadata!,Is empty!,Is empty!,Is empty!,Using 6 IKEA AA Ni-MH batteries; Used Acoustic...,0.03,6.873080,8
10011,2022-08-02 21:00:00,Carp Pond,recover-20220804,C,001,/mnt/ubna_data_01/recover-20220804/UBNA_001/20...,Recorded at 21:00:00 02/08/2022 (UTC) by Audio...,Usable for detection,23.9C,4.0V,192000,AudioMoth 24F319055FDF2F5B,1795,Double Battery Pack; Ziploc Bag,0.00,9.035484,8
19279,2022-09-05 21:00:00,Carp Pond,recover-20220905,C,10,/mnt/ubna_data_02/recover-20220905/UBNA_010/20...,Is empty!,Not usable; no metadata,File does not have metadata!,File does not have metadata!,Is empty!,Is empty!,Is empty!,Ikea; Heat is dying down; Rained during sessio...,0.05,8.232284,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28678,2022-10-11 06:30:00,Telephone Field,recover-20221013,A,1,/mnt/ubna_data_02/recover-20221013/UBNA_001/20...,Recorded at 06:30:00 11/10/2022 (UTC) by Audio...,Usable for detection,20.7C,4.0V,192000,AudioMoth 24F3190361CBE990,1795.00127083333,IKEA,0.00,4.935484,10
47827,2022-11-25 18:00:00,Telephone Field,recover-20221208,A,7,/mnt/ubna_data_02/recover-20221208/UBNA_007/20...,Recorded at 18:00:00 25/11/2022 (UTC) by Audio...,Usable for detection,11.4C,4.1V,192000,AudioMoth 24F3190361CBE990,300.001270833333,Panasonic,0.00,NaN,11
40270,2022-11-07 21:00:00,Telephone Field,recover-20221110,A,7,/mnt/ubna_data_02/recover-20221110/UBNA_007/20...,Recorded at 21:00:00 07/11/2022 (UTC) by Audio...,Usable for detection,9.9C,3.9V,192000,AudioMoth 24F3190361CBE990,300.001270833333,Panasonic,0.00,NaN,11
49542,2022-12-02 21:30:00,Telephone Field,recover-20221208,A,7,/mnt/ubna_data_02/recover-20221208/UBNA_007/20...,Recorded at 21:30:00 02/12/2022 (UTC) by Audio...,Usable for detection,9.3C,4.0V,192000,AudioMoth 24F3190361CBE990,300.001270833333,Panasonic,0.00,NaN,12


Save the generated randomized dataset in the folder location.

In [8]:
selected_rows.to_csv('selection_under_randomization_file.csv', index=False)